<a href="https://colab.research.google.com/github/Domineice/Bigdata-class/blob/main/Exe_Dataframe1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 51.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=9dd62f0e527d26cfb57cec28a11e537f56483e805e3bc88e14b9bfde99bd97df
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
import findspark
findspark.init()


In [ ]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [ ]:
# Creating a spark context class
sc = SparkContext()

In [ ]:
spark = SparkSession \
    .builder \
    .getOrCreate()

In [ ]:
data=[('A',1),('B',2),('C',3)]

In [ ]:
rdd=sc.parallelize(data)

In [ ]:
print(rdd.collect())

[('A', 1), ('B', 2), ('C', 3)]


# Create DataFrame

In [ ]:
dfFromRDD=rdd.toDF()
dfFromRDD.printSchema()
dfFromRDD.show()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)

+---+---+
| _1| _2|
+---+---+
|  A|  1|
|  B|  2|
|  C|  3|
+---+---+



In [ ]:
header=['word','count']

In [ ]:
dfFromRDD1=rdd.toDF(header)
dfFromRDD1.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = true)



In [ ]:
dfFromRDD1.show()

+----+-----+
|word|count|
+----+-----+
|   A|    1|
|   B|    2|
|   C|    3|
+----+-----+



Option2: Import RDD with column

In [ ]:
dfFromRDD2=spark.createDataFrame(rdd).toDF(*header)

In [ ]:
dfFromRDD2.printSchema()

root
 |-- key: string (nullable = true)
 |-- value: long (nullable = true)



In [ ]:
dfFromRDD2.show()

+---+-----+
|key|value|
+---+-----+
|  A|    1|
|  B|    2|
|  C|    3|
+---+-----+



Option3: Import the data directly

In [ ]:
dfFromRDD3=spark.createDataFrame(data).toDF(*header)
dfFromRDD3.printSchema()
dfFromRDD3.show()

root
 |-- key: string (nullable = true)
 |-- value: long (nullable = true)

+---+-----+
|key|value|
+---+-----+
|  A|    1|
|  B|    2|
|  C|    3|
+---+-----+



In [ ]:
sc.stop()

# Import DataFrame

In [ ]:
#Set directory
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
path = '/content/drive/My Drive/bigdata/'

Mounted at /content/drive/


In [ ]:
#Reading csv files

In [ ]:
# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames Import example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

NameError: ignored

In [ ]:
dataframe = spark.read.csv(path+"students.csv")


In [ ]:
dataframe.show()

In [ ]:
dataframe = spark.read.csv(path+"students.csv",header='true')
dataframe.show()

In [ ]:
dataframe.printSchema()


In [ ]:
dataframe = spark.read.csv(path+"students.csv",header='true',inferSchema=True)
dataframe.show()
dataframe.printSchema()


In [ ]:
#Get Column Name
dataframe.columns

In [ ]:
# Read the dataset into a spark dataframe using the `read.json()` function
df = spark.read.json(path+"people.json").cache()

In [ ]:
# Print the dataframe as well as the data schema
df.show()


In [ ]:
df.printSchema()

**Task 2**:** Explore the data using DataFrame** functions and SparkSQL




In this section, we explore the datasets using functions both from dataframes as well as corresponding SQL queries using sparksql. Note the different ways to achieve the same task!# New Section

In [ ]:
# Select and show basic data columns
df.select("age").show()
df.select(df["name"]).show()


In [ ]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")


In [ ]:
spark.sql("SELECT name,age FROM people").show()

In [ ]:
df.sort("age").show()
spark.sql("SELECT * FROM people order by age desc").show()

In [ ]:
df.withColumnRenamed('name','Students_name').show()

In [ ]:
df.withColumnRenamed('Students_name','name').show()

In [ ]:
df_new=df.withColumn('Graduation age',df['age']+1)
df_new.show()


In [ ]:
df_new.drop('Graduation age').show()

In [ ]:
# Perform basic filtering

df.filter(df["age"] > 21).show()
spark.sql("SELECT * FROM people WHERE age > 21").show()

In [ ]:
# Perfom basic aggregation of data

df.groupBy("age").count().show()
spark.sql("SELECT age, COUNT(age) as count FROM people GROUP BY age").show()

# Exercise 1



 **Exercise1** read the people2.json file into the notebook, load it into a dataframe and apply SQL operations to determine the average age in our people2 file.

In [ ]:
#Exercise 1
# df = spark.read...
df1 = spark.read.json(path+"people2.json")
# df.createTempView..
# spark.sql("SELECT ...")
from pyspark.sql import functions as f
df1.createOrReplaceTempView("student")


**Exercise2** read the people2.json file into the notebook, load it into a dataframe and apply SQL operations to determine NUMBER OF PEOPLE IN EACH PROGRAM/ AVERAGE  IN EACH PROGRAM


In [ ]:
df1.groupby('major').agg(f.count('*'),f.avg('age')).show()

+------+--------+--------+
| major|count(1)|avg(age)|
+------+--------+--------+
|EGCO/M|       5|   24.75|
|  EGCO|       2|    22.0|
+------+--------+--------+



In [ ]:
from pyspark.sql.functions import mean
df1.na.fill(df1.select(f.mean(df['age'])).collect()[0][0],['age'])

df1.show()

+------+----+-----------+
| Major| age|       name|
+------+----+-----------+
|EGCO/M|  30|  Nartdanai|
|EGCO/M|  29|      Tanut|
|EGCO/M|null|  Nattapark|
|EGCO/M|  20|  Weerawich|
|  EGCO|  22|Threerapong|
|EGCO/M|  20|  Weerawich|
|  EGCO|  22|Threerapong|
+------+----+-----------+



In [ ]:
merge.show()

+----+---+---+-----+---+
|name|age|gpa|Major|age|
+----+---+---+-----+---+
+----+---+---+-----+---+



In [ ]:
def UpperCase(str):
  return str.upper()

In [ ]:
from pyspark.sql.functions import udf

upperCaseUDF=udf(lambda z: UpperCase(z))

In [ ]:
df2=df1.withColumn("Uppercase Name", upperCaseUDF("name"))

In [ ]:
df2.show()

+------+----+-----------+--------------+
| Major| age|       name|Uppercase Name|
+------+----+-----------+--------------+
|EGCO/M|  30|  Nartdanai|     NARTDANAI|
|EGCO/M|  29|      Tanut|         TANUT|
|EGCO/M|null|  Nattapark|     NATTAPARK|
|EGCO/M|  20|  Weerawich|     WEERAWICH|
|  EGCO|  22|Threerapong|   THREERAPONG|
|EGCO/M|  20|  Weerawich|     WEERAWICH|
|  EGCO|  22|Threerapong|   THREERAPONG|
+------+----+-----------+--------------+



In [ ]:

def std(input,mean,sd):
  return (input-mean)/sd

stdudf=udf(lambda )


In [ ]:
df2=df1.withColumn("std", stdudf('age',10,20))

NameError: ignored

**Exercise3** Remove Duplicate

In [ ]:
#Exercise 3

**Exercise4** REMOVE NULL VALUE or use average value to fill in the NULL

In [ ]:
#Exercise 4

In [ ]:
#close session
spark.stop()

**Caching Data**
This simple example shows how to create an RDD and cache it. Notice the 10x speed improvement! If you wish to see the actual computation time.

In [ ]:
import time 

test = sc.parallelize(range(1,50000),4)
test.cache()

t1 = time.time()
# first count will trigger evaluation of count *and* cache
count1 = test.count()
dt1 = time.time() - t1
print("dt1: ", dt1)


t2 = time.time()
# second count operates on cached data only
count2 = test.count()
dt2 = time.time() - t2
print("dt2: ", dt2)

#test.count()

dt1:  0.1692793369293213
dt2:  0.08459901809692383
